In [1]:
from pymongo import MongoClient
import pandas as pd
import json
import cleaning as f

# Data cleaning

#### Se hace una pequeña limpieza y primeros filtros para las siguientes tomas de decisiones

- Importamos base de datos de compañías

In [2]:
client = MongoClient("mongodb://localhost/companies")

In [3]:
db = client.get_database()

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

- Filtramos por empresas vivas y convertimos en DataFrame para poder operar

In [5]:
query = {'deadpooled_year':{'$eq':None}}

In [6]:
data = list(db["companies"].find(query,{"_id":0,"name":1,"category_code":1,"number_of_employees":1,"founded_year":1,"deadpooled_year":1,"total_money_raised":1,"offices":1}))

df = pd.DataFrame(data)

- Aplicamos la función que expande la columna de 'offices' y genera una nueva columna 'location' con el formato correcto de las coordenadas

In [7]:
df2 = f.officesClean(df)

In [8]:
df2.head()

,name,category_code,number_of_employees,founded_year,deadpooled_year,total_money_raised,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,location
0,Postini,web,NaN,1999.0,None,$0,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573,"{'type': 'Point', 'coordinates': [-122.247573,..."
1,Digg,news,60.0,2004.0,None,$45M,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,Flektor,games_video,NaN,NaN,None,$0,None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768,"{'type': 'Point', 'coordinates': [-118.379768,..."
3,Fox Interactive Media,web,0.0,1979.0,None,$0,,407 N Maple Dr,,90210,Beverly Hills,CA,USA,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ..."
4,Geni,web,18.0,2006.0,None,$16.5M,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."


- Exportamos como json para hacer consultas/modificar información

In [ ]:
df2.to_json("../OUTPUT/first_order", orient="records")

- Importamos de nuevo el primer json limpio y filtramos solo por localizaciones con coordenadas

In [9]:
query2 = {'location':{'$ne':None}}

In [10]:
data2 = list(db["first_order"].find(query2,{"_id":0}))

first_order = pd.DataFrame(data2)

- Categorizamos las compañías según actividad para quedarnos solo con las tecnológicas

In [11]:
tech_comp = ["web","software","mobile","games_video","ecommerce","network_hosting","hardware","biotech","cleantech","analytics","semiconductor","photo_video","software","nanotech"]

first_order["tech_company"] = df["category_code"].apply(lambda x: "yes" if x in tech_comp else "no")

- Comprobamos que la mayoría de las empresas tecnológicas se sitúan en San Francisco, por tanto, ésta será la ciudad en la que buscaremos donde posicionarnos

In [12]:
tech = first_order['tech_company'] == 'yes'
filt = first_order[tech]

filt.city.value_counts().head(5)

San Francisco    399
New York         335
London           184
                 153
Seattle           90
Name: city, dtype: int64

- Exportamos de nuevo el fichero ya limpio con todas las coordenadas y solo empresas con actividad en el área tecnológica

In [ ]:
first_order.to_json("../OUTPUT/cleaned_companies", orient="records")